In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from tqdm import tqdm
from sklearn.decomposition import PCA

In [2]:
motor_arr= [
    '03_08/VIB_0308_motor_51200.csv',
    '03_10/VIB_0310_motor_51200.csv',
    '03_11/VIB_0311_motor_51200.csv',
    '03_14/VIB_0314_motor_51200.csv',
    '03_16/VIB_0316_motor_51200.csv',
    '03_17/VIB_0317_motor_51200.csv',
    '03_21/VIB_0321_motor_51200.csv',
    '03_23/VIB_0323_motor_51200.csv',
    '03_24/VIB_0324_motor_51200.csv',
    '03_28/VIB_0328_motor_51200.csv'
]

In [3]:
pump_arr= [
#     '03_08/VIB_0308_pump_51200.csv',
#     '03_10/VIB_0310_pump_51200.csv',
#     '03_11/VIB_0311_pump_51200.csv',
    '03_14/VIB_0314_pump_51200.csv',
    '03_16/VIB_0316_pump_51200.csv',
    '03_17/VIB_0317_pump_51200.csv',
    '03_21/VIB_0321_pump_51200.csv',
    '03_23/VIB_0323_pump_51200.csv',
    '03_24/VIB_0324_pump_51200.csv',
    '03_28/VIB_0328_pump_51200.csv'
]

In [4]:
def pca(train_data,dimension = 2):
    pca = PCA(n_components=dimension)
    printcipalComponents = pca.fit_transform(train_data)
    principalDf = pd.DataFrame(data=printcipalComponents, columns = ['x1', 'x2'])
    return principalDf

In [5]:
def get_features(train_data):
    describe = train_data.describe(include='all')
    mean = describe.loc['mean']
    std = describe.loc['std'] #표준편차,
    min = describe.loc['min'] 
    first_quartile= describe.loc['25%']
    median = describe.loc['50%']
    third_quartile = describe.loc['75%']
    max = describe.loc['max'] 
    
    return [[mean,std,min,first_quartile,median,third_quartile,max]]

In [6]:
def data_process(data_frame,batch_size=5110):
    before = 0
    train_data = pd.DataFrame([[]])
    for i in tqdm(range(batch_size,len(data_frame),batch_size)):
        features = pd.DataFrame(get_features(data_frame[before:i]),
                                columns = ['mean','std','min','first_quartile','median','third_quartile','max'])
        before = i
        train_data = train_data.append(features,ignore_index=True)
        
    return train_data[1:]

In [7]:
def collecting(arr):
    collected_data = pd.DataFrame([[0,0]],columns = ['x1','x2'])
    for filename in tqdm(arr):
        data_set = pd.read_csv(filename, encoding = 'cp949',sep = '\t')
        data_set = data_process(data_set.iloc[:47960000,1])
        data_set = pca(data_set)
        collected_data = collected_data.append(data_set)
    return collected_data

In [8]:
def get_abnormal_data(dataset,batch_size=5110):
    minv= dataset.iloc[:,1].min()
    maxv = dataset.iloc[:,1].max()
    rng = np.random.RandomState(42)
    before = 0
    train_data = pd.DataFrame([[0]],columns = ['x1'])
    for i in tqdm(range(batch_size,len(dataset),batch_size)):  
        fragment = rng.uniform(low=minv - 10, high=maxv + 10, size=batch_size)
        fragment = pd.DataFrame(fragment,columns = ['x1'])
        
        before = i
        train_data = train_data.append(fragment)
        
    return train_data

In [9]:
#collected_data = collecting(pump_arr)

In [10]:
#collected_data.iloc[1:].to_csv('pump_collected.csv',encoding='cp949')

In [11]:
index = 4
for filename in pump_arr:
        print(index,'번째 파일 읽는중.............................')
        data_set = pd.read_csv(filename, encoding = 'cp949',sep = '\t')
        print(index,'번째 고장 데이터 생성.............................')
        data_set = get_abnormal_data(data_set)
        data_set.to_csv("pump_abnormal"+str(index) +'.csv',encoding='cp949',index=False)
        index += 1

4 번째 파일 읽는중.............................
4 번째 고장 데이터 생성.............................


100%|██████████████████████████████████████████████████████████████████████████| 25017/25017 [1:46:53<00:00,  3.90it/s]


5 번째 파일 읽는중.............................
5 번째 고장 데이터 생성.............................


 47%|███████████████████████████████████▍                                        | 13044/27979 [32:10<36:49,  6.76it/s]


KeyboardInterrupt: 